In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-17 16:49:52,222 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 16:49:52,227 - settings - Setting database.user to celiib
INFO - 2021-05-17 16:49:52,227 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 16:49:52,230 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-17 16:49:52,231 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 16:49:52,240 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-17 16:49:52,656 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 16:49:52,660 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 16:49:52,662 - settings - Setting database.user to celiib
INFO - 2021-05-17 16:49:52,663 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 16:49:52,663 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 16:49:52,664 - settings - Setting database.user to celiib
INFO - 2021-05-17 16:49:52,665 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 16:49:52,666 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 16:49:53,105 - settings - Setting enable_python_native_blobs to True


In [3]:
import datajoint as dj
version = 88
nucleus_table_ids = du.nucleus_info_table(version = 88) & "segment_id>0"
minnie, schema = du.configure_minnie_vm()


INFO - 2021-05-17 16:49:53,234 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-17 16:49:53,235 - settings - Setting database.user to celiib
INFO - 2021-05-17 16:49:53,236 - settings - Setting database.password to newceliipass
INFO - 2021-05-17 16:49:53,240 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-17 16:49:53,733 - settings - Setting enable_python_native_blobs to True


In [4]:
du.nucleus_info_table(version = 88) & "segment_id>0"
nucleus_table_ids.proj("segment_id")

ver materialization version,nucleus_id id of nucleus from the flat segmentation Equivalent to Allen: 'id'.,segment_id id of the segment under the nucleus centroid. Equivalent to Allen 'pt_root_id'.
88.00,1998,864691136050815731
88.00,2198,864691135084884330
88.00,2199,864691136521572241
88.00,2389,864691135494586958
88.00,2398,864691136451074559
88.00,2433,864691135524436073
88.00,3024,864691135087399350
88.00,3117,864691134684964653
88.00,3121,864691132625568958
88.00,3152,864691135524409705


In [19]:
minnie.NucleiSegmentsRun2.describe()

segment_id           : bigint unsigned              # segment id for those to be decimated



'segment_id           : bigint unsigned              # segment id for those to be decimated\n'

In [24]:
@schema
class NucleiSegmentsRun4(dj.Manual):
    definition="""
    segment_id           : bigint unsigned              # segment id for those to be decimated
    ---
    """
keys_to_insert = du.version_to_segment_id_table().fetch(as_dict=True)
NucleiSegmentsRun4.insert(keys_to_insert,skip_duplicates=True)


In [ ]:
minnie, schema = du.configure_minnie_vm()
changed_ids = nucleus_table_ids.proj() - minnie.Segment
changed_ids

In [ ]:
changed_ids = nucleus_table_ids.proj() - minnie.Mesh
changed_ids

In [ ]:
need_to_download_mesh_ids = changed_ids.fetch("segment_id")
need_to_download_mesh_ids.shape

In [ ]:
import numpy as np
np.savez("v88_mesh_ids_to_download.npz",segment_ids=need_to_download_mesh_ids)

In [ ]:
meshes_to_down = np.load("v88_mesh_ids_to_download.npz")["segment_ids"]
meshes_to_down

In [ ]:
need_to_download_mesh_ids.shape

In [ ]:
np.unique(need_to_download_mesh_ids).shape

In [ ]:
meshes_to_down.shape

In [ ]:
s = 864691136939680735
Path(f"/mnt/dj-stor01/platinum/minnie65/02/meshes/{s}.h5").exists()

# Trying and Filling in the IDs

In [ ]:
du = reload(du)

In [ ]:
import os
external_mesh_path = du.get_external_mesh_path()

external_mesh_path
@schema
class Mesh(dj.Imported):
    definition = """
    # Platinum mesh structures
    -> Segment
    ---
    n_vertices          : int unsigned       # number of vertices
    n_faces             : int unsigned       # number of faces
    mesh                : <mesh>             # in-place path to the hdf5 mesh file
    """

    @classmethod
    def fill_from_ids(cls, segment_ids, skip_duplicates=True, skip_missing=True, extra_skip_missing=True, disable_tqdm=False, suppress_errors=True):
        if skip_missing:
            if extra_skip_missing:
                segment_ids = frozenset(segment_ids).difference(cls.fetch('segment_id'))
            remaining_segments = []
            for segment_id in tqdm(segment_ids, disable=disable_tqdm):
                if os.path.isfile(external_mesh_path / f'{segment_id}.h5'):
                    remaining_segments.append(segment_id)
            segment_ids = remaining_segments

        
        for segment_id in tqdm(segment_ids, disable=disable_tqdm):
            mesh_path = os.path.join(external_mesh_path, f'{segment_id}.h5')
            print(f"mesh_path = {mesh_path}")
            #mesh_lengths = du.adapt_mesh_hdf5(filepath=mesh_path, as_lengths=True, suppress_errors=suppress_errors)
            mesh_lengths = du.adapt_mesh_hdf5(filepath=mesh_path, as_lengths=True,)# suppress_errors=suppress_errors)
            if mesh_lengths is not None:
                n_vertices, n_faces = mesh_lengths
                cls.insert1(dict(
                    segment_id=segment_id,
                    n_vertices=n_vertices,
                    n_faces=n_faces,
                    mesh=mesh_path
                    ),
                    skip_duplicates=skip_duplicates,
                    allow_direct_insert=True
                )

                
with np.load('v88_mesh_ids_to_download.npz') as f:
    mesh_ids = f['segment_ids']

Mesh.fill_from_ids([need_to_download_mesh_ids[100]], skip_missing=False)

In [ ]:
du.insert_segment_ids_and_meshes(restr_table.fetch("segment_id"))

In [ ]:
minnie.Mesh.describe()

In [10]:
m65mat = dj.create_virtual_module('m65mat', 'microns_minnie65_materialization')
nucleus_table_ids = m65mat.Nucleus.Info & {'ver': 88} & "segment_id>0"
seg_ids_v88 = dj.U("segment_id") & nucleus_table_ids.proj("segment_id")
restr_table = seg_ids_v88 - minnie.Mesh.proj()
restr_table

from pathlib import Path

s_exists = []
for s in restr_table.fetch("segment_id"):
    if Path(f"/mnt/dj-stor01/platinum/minnie65/02/meshes/{s}.h5").exists():
        s_exists.append(s)
        
du.insert_segment_ids_and_meshes(np.array(s_exists[1:]),suppress_errors = False,
                                insert_into_Segment=False)

[864691137053796982, 864691135403892718]

In [48]:
decimation_version = 0
decimation_ratio_global = 0.25
(du.proofreading_segment_id_restriction() - (minnie.Decimation() & dict(decimation_ratio=decimation_ratio_global)).proj()) & minnie.Mesh.proj()

segment_id segment id for those to be decimated
864691135181843970
864691135210286272
864691135278405537
864691135291790518
864691135323105180
864691135323123100
864691135341221573
864691135341243057
864691135341277105
864691135367342841


In [43]:
curr_table = (minnie.schema.jobs & "table_name='__decimation'")
curr_table.delete()

In [49]:
minnie.DecompositionAxon()

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,axon_version key by which to lookup the finer axon processing method,decomposition saved neuron object with high fidelity axon,axon_length length (in um) of the classified axon skeleton,run_time the amount of time to run (seconds)
864691134219060901,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.2
864691134221889045,0,0.25,30.00,5,0,0,4,=BLOB=,0.0,1.16
864691134242673681,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.33
864691134252908410,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.14
864691134316523453,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.95
864691134384033872,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.26
864691134391593458,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.18
864691134409056935,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.38
864691134428609977,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.13
864691134441691943,0,0.25,30.00,0,0,0,4,=BLOB=,0.0,0.56
